In [7]:
from utils import TestResults,TrainSplit
from sklearn.metrics import log_loss

import lightgbm as lgb
import pandas as pd
import numpy as np
import optuna
# from tqdm import tqdm
# ROWID = ['f_0']
# DATE = ['f_1']
# CATEGORIES = [ f'f_{i}' for i in range(2,33) ]
# BINARY = [ f'f_{i}' for i in range(33,42) ]
# NUMERICAL = [ f'f_{i}' for i in range(42,80) ]
# IS_CLICKED = ['is_clicked']
# IS_INSTALLED =['is_installed']

In [3]:
X_train, X_test, y_train, y_test = TrainSplit().get_split()

Loading the data
Categorial Feature Imputed


NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 54.18it/s]


Spliting the Data based on time
X_train:(3387880, 80), X_test:(97972, 80) , y_train:(3387880, 2) , y_test:(97972, 2)


In [4]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [10]:
def objective(trail):
    params = {
        'max_depth':trail.suggest_int('max_depth',3,8),
        'learning_rate':trail.suggest_float('learning_rate',0.05,0.3),
        'num_boost_round':trail.suggest_int('num_boost_round',100,650),
        'num_leaves':'31',
        'objective':'binary'
    }
    # print(use_features)
    model = lgb.train(params,data)
    y_pred = model.predict(X_test[use_features])
    score = log_loss(y_test[ TrainSplit.IS_INSTALLED[0]],y_pred)
    # print(model.feature_names_in_)
    return score

In [ ]:
use_features = all_feat
data = lgb.Dataset(X_train[use_features],label=y_train[ TrainSplit.IS_INSTALLED[0]])
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-24 22:20:31,408] A new study created in memory with name: install_log_loss
/home2/sisodiya.bhoomendra/venvs/python3.9_global/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


## Imputation

In [4]:
missing = pd.read_csv('../Data/miss_combine.csv')
missing['f_30'].fillna(missing['f_30'].mode()[0],inplace=True)
missing['f_31'].fillna(missing['f_31'].mode()[0],inplace=True)
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in tqdm(fmiss,desc="NUM IMPUTE"):
    missing[f].fillna(missing[f].mean(),inplace=True)

NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 51.65it/s]


In [5]:
X = missing[user_feat]

In [6]:
click_train_data = lgb.Dataset(X,label=missing[IS_CLICKED])

In [12]:
params = {'objective':'cross_entropy',
         "verbose":2}

In [16]:
click_model = lgb.train(params,click_train_data,num_boost_round=500)

[LightGBM] [Info] [cross_entropy:Init]: (objective) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: (metric) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: sum-of-weights = 3485852.000000
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.927215
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.468756
[LightGBM] [Debug] init for col-wise cost 0.225457 seconds, init for row-wise cost 0.696564 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.285477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 5840
[LightGBM] [Info] Number of data points in the train set: 3485852, number of used features: 77
[LightGBM] [Info] [cross_entropy:BoostFromScore]: pavg = 0.219835 -> initscore = -1.266625
[Li

In [17]:
install_train_data = lgb.Dataset(X,label=missing[IS_INSTALLED])

In [18]:
install_model = lgb.train(params,install_train_data,num_boost_round=500)

[LightGBM] [Info] [cross_entropy:Init]: (objective) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: (metric) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: sum-of-weights = 3485852.000000
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.927215
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.468756
[LightGBM] [Debug] init for col-wise cost 0.198393 seconds, init for row-wise cost 0.636647 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.263844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 5840
[LightGBM] [Info] Number of data points in the train set: 3485852, number of used features: 77
[LightGBM] [Info] [cross_entropy:BoostFromScore]: pavg = 0.174018 -> initscore = -1.557412
[Li

In [19]:
test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')

In [20]:
test['f_30'].fillna(test['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(test['f_31'].mode()[0],inplace=True)
for f in fmiss:
    test[f].fillna(test[f].mean(),inplace=True)

In [21]:
click_pred = click_model.predict(test[user_feat])

In [25]:
install_pred = install_model.predict(test[user_feat])

In [28]:
import numpy as np
result = np.vstack([test['f_0'],click_pred,install_pred]).T

In [29]:
result.shape

(160973, 3)

In [30]:
final = pd.DataFrame(result,columns=['RowId','is_clicked','is_installed'])
final.head()

,RowId,is_clicked,is_installed
0,64505.0,0.332214,0.159798
1,64506.0,0.094155,0.252163
2,64507.0,0.235014,0.052124
3,64508.0,0.119201,0.189894
4,64509.0,0.236937,0.177585


In [31]:
final['RowId']=final['RowId'].astype('int')
final.to_csv('../Data/final_results/lightbgm_result_n_estimator_500.csv', sep ='\t', index=False)